# Data Preprocessing

In [10]:
import pandas as pd
import numpy as np

data = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv")
data.head()

      year       imdb                                    title  \
0     2013  tt1711425                            21 &amp; Over   
1     2012  tt1343727                                 Dredd 3D   
2     2013  tt2024544                         12 Years a Slave   
3     2013  tt1272878                                   2 Guns   
4     2013  tt0453562                                       42   
...    ...        ...                                      ...   
1789  1971  tt0067741                                    Shaft   
1790  1971  tt0067800                               Straw Dogs   
1791  1971  tt0067116                    The French Connection   
1792  1971  tt0067992  Willy Wonka &amp; the Chocolate Factory   
1793  1970  tt0065466           Beyond the Valley of the Dolls   

                 test clean_test binary    budget    domgross     intgross  \
0              notalk     notalk   FAIL  13000000  25682380.0   42195766.0   
1         ok-disagree         ok   PASS  45000000  

In [3]:
select_columns = data[['year', 'imdb', 'title', 'clean_test', 'binary', 'budget_2013$', 'domgross_2013$', 'intgross_2013$']]

In [4]:
select_columns.head()

,year,imdb,title,clean_test,binary,budget_2013$,domgross_2013$,intgross_2013$
0,2013,tt1711425,21 &amp; Over,notalk,FAIL,13000000,25682380.0,42195766.0
1,2012,tt1343727,Dredd 3D,ok,PASS,45658735,13611086.0,41467257.0
2,2013,tt2024544,12 Years a Slave,notalk,FAIL,20000000,53107035.0,158607035.0
3,2013,tt1272878,2 Guns,notalk,FAIL,61000000,75612460.0,132493015.0
4,2013,tt0453562,42,men,FAIL,40000000,95020213.0,95020213.0


In [5]:
select_columns.corr()

,year,budget_2013$,domgross_2013$,intgross_2013$
year,1.000000,0.077695,-0.286437,-0.135760
budget_2013$,0.077695,1.000000,0.461261,0.594019
domgross_2013$,-0.286437,0.461261,1.000000,0.927652
intgross_2013$,-0.135760,0.594019,0.927652,1.000000


In [6]:
select_columns.cov()

,year,budget_2013$,domgross_2013$,intgross_2013$
year,8.063557e+01,3.831578e+07,-3.232021e+08,-3.443421e+08
budget_2013$,3.831578e+07,3.016057e+15,3.196803e+15,9.258982e+15
domgross_2013$,-3.232021e+08,3.196803e+15,1.586727e+16,3.316561e+16
intgross_2013$,-3.443421e+08,9.258982e+15,3.316561e+16,8.037676e+16


In [7]:
# Check for duplicates: True if duplicates were found, False if all data is unique
def check_duplicates(data):
    if len(select_columns['imdb']) != len(select_columns['imdb'].unique()):
        return True
    else:
        return False

In [21]:
# Check for null values: return columns and sums of null values located in each
def null_values_in_columns(data):
    return data.isnull().sum()
print(null_values_in_columns(select_columns))

year              0
imdb              0
title             0
clean_test        0
binary            0
budget_2013$      0
domgross_2013$    0
intgross_2013$    0
dtype: int64


In [22]:
select_columns = select_columns[select_columns['domgross_2013$'].isnull() == False][select_columns['intgross_2013$'].isnull() == False]

In [20]:
print(null_values_in_columns(select_columns))

year              0
imdb              0
title             0
clean_test        0
binary            0
budget_2013$      0
domgross_2013$    0
intgross_2013$    0
dtype: int64

In [ ]:
#Gather TMDB data